## ETL Wine

### Initial configuration

In [0]:
dbutils.library.restartPython()

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType, ArrayType
from pyspark.sql.functions import size,split, regexp_replace, expr, lower, array_except, col, lit, array, transform, trim, filter, udf

### Load wine dataset in a Dataframe

In [0]:
raw_wine = spark.read.csv(
    "/Volumes/wine_harmonization/datasets/raw_datasets/WineDataset.csv",
    header=True,
    inferSchema=True,
    escape='"'
)

In [0]:
display(raw_wine.count())
display(raw_wine)

In [0]:
exclusion = spark.read.csv(
    "/Volumes/wine_harmonization/datasets/raw_datasets/unique_flavors.csv",
    header=True,
    inferSchema=True,
    comment='#'
)

exclusion = exclusion.withColumn("flavor", lower(exclusion["flavor"])).dropDuplicates(subset=["flavor"])
display(exclusion)

### Clean rows

In [0]:
wines_clean = raw_wine.drop(
    "Appellation", "Vintage", "Style", "Region", "ABV", "Description", "Closure", "Secondary Grape Varieties", "Unit", "Per bottle / case / each", "Capacity", "Country"
).dropna(subset=["Title", "Grape", "Characteristics", "Price", "Type"])
display(wines_clean)

In [0]:
from pyspark.sql import functions as F

wines_filtered = wines_clean.filter(
    F.col("Type").isin(["Red", "White", "Rosé"])
)

display(wines_filtered)

In [0]:
wines_clean = wines_clean.withColumnRenamed("_c0", "id").withColumnRenamed("NER", "Characteristics")
display(wines_clean)

In [0]:
exclusion_words = [row['flavor'] for row in exclusion.select("flavor").distinct().collect()]

def remove_excluded_words(Characteristics, exclusion_set):
    if not Characteristics:
        return []
    
    result = []
    for ingredient in Characteristics:
        if ingredient:
            words = ingredient.split()
            filtered_words = [word for word in words if word in exclusion_set]
            filtered_ingredient = ' '.join(filtered_words).strip()
            
            if filtered_ingredient:
                result.append(filtered_ingredient)
    
    return result

exclusion_set = set(exclusion_words)

remove_excluded_udf = udf(lambda ingredients: remove_excluded_words(ingredients, exclusion_set), ArrayType(StringType()))

wines_clean_test = (
    wines_filtered
    .withColumn("Characteristics", regexp_replace("Characteristics", r'[\[\]\"]', ''))
    .withColumn("Characteristics", split("Characteristics", ","))
    .withColumn("Characteristics", expr("transform(Characteristics, x -> lower(trim(x)))"))
    .withColumn("Characteristics", remove_excluded_udf(col("Characteristics")))
)
display(wines_clean_test)

In [0]:
from pyspark.sql import functions as F

# Supondo que seu DataFrame seja wines_clean e a coluna se chame "Characteristics"
wines_no_dups = wines_clean_test.withColumn(
    "Characteristics",
    F.array_distinct(F.col("Characteristics"))
)

display(wines_no_dups)

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import FloatType

# Cotação GBP→BRL (ajuste conforme necessário)
gbp_to_brl = 7.44

wines_final = (
    wines_no_dups
    # normalização e extração do float
    .withColumn(
        "price_str",
        F.when(F.col("Price").startswith("£"), F.col("Price"))
         .otherwise(None)
    )
    .withColumn("price_norm",    F.regexp_replace("price_str", ",", "."))
    .withColumn("price_clean",   F.regexp_replace("price_norm", "[^0-9\\.]", ""))
    .withColumn("price_float",   F.col("price_clean").cast(FloatType()))
    # cálculo e arredondamento em reais
    .withColumn("price_brl",     F.round(F.col("price_float") * F.lit(gbp_to_brl), 2))
    # remove todas as colunas intermediárias e a original
    .drop("Price", "price_str", "price_norm", "price_clean", "price_float")
    # renomeia price_brl para Price (ou outro nome que preferir)
    .withColumnRenamed("price_brl", "Price")
)

display(wines_final)


### Save in a table

In [0]:
wines_final.write.mode("overwrite").saveAsTable("wine_harmonization.datasets.wines")